In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Module 1: Wikipedia Ingestion

from typing import List
import spacy


# ---------- 1. Loader ----------
def load_raw_wikipedia(path: str) -> str:
    """
    Load the raw Simple Wikipedia text file.
    """
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()
    return text


# ---------- 2. Normalizer ----------
def normalize_text(text: str) -> str:
    """
    Normalize whitespace without changing semantics.
    """
    lines = text.splitlines()
    cleaned_lines = []

    for line in lines:
        line = line.strip()
        if line:
            cleaned_lines.append(line)

    return "\n".join(cleaned_lines)


# ---------- 3. Sentence Splitter ----------
def split_into_sentences(text: str, batch_size: int = 100_000) -> List[str]:
    """
    Split very large text into sentences safely by processing in chunks.
    """
    nlp = spacy.load(
        "en_core_web_sm",
        disable=["ner", "tagger", "lemmatizer"]
    )

    sentences = []

    for i in range(0, len(text), batch_size):
        text_slice = text[i:i + batch_size]
        doc = nlp(text_slice)

        for sent in doc.sents:
            sent_text = sent.text.strip()
            if sent_text:
                sentences.append(sent_text)

    return sentences

# ---------- 4. Public Interface ----------
def ingest_wikipedia(path: str) -> List[str]:
    """
    End-to-end ingestion pipeline.
    """
    raw_text = load_raw_wikipedia(path)
    normalized_text = normalize_text(raw_text)
    sentences = split_into_sentences(normalized_text)
    return sentences

In [ ]:
sentences = ingest_wikipedia("/content/sample_data/AllCombined.txt")
print(len(sentences))
print(sentences[0])

149749
April
April (Apr.) is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May.


In [ ]:
import pickle

sentences = ingest_wikipedia("/content/sample_data/AllCombined.txt")

with open("/content/drive/MyDrive/wiki_sentences.pkl", "wb") as f:
    pickle.dump(sentences, f)


In [ ]:
with open("/content/drive/MyDrive/wiki_sentences.pkl", "rb") as f:
    sentences = pickle.load(f)

print(len(sentences))

149749


In [ ]:
from typing import List, Dict
import pickle

def count_tokens(text: str) -> int:
    """
    Approximate token count using word count.
    1 token ≈ 0.75 words (English heuristic).
    """
    return int(len(text.split()) / 0.75)

def build_chunks(
    sentences: List[str],
    min_tokens: int = 300,
    max_tokens: int = 600,
    overlap_tokens: int = 100
) -> List[Dict]:
    """
    Build sentence-aware overlapping chunks.
    """

    chunks = []
    current_sentences = []
    current_tokens = 0
    chunk_id = 0

    for sentence in sentences:
        sentence_tokens = count_tokens(sentence)

        # If adding this sentence exceeds max_tokens, finalize chunk
        if current_tokens + sentence_tokens > max_tokens:
            if current_tokens >= min_tokens:
                chunk_text = " ".join(current_sentences)
                chunks.append({
                    "chunk_id": chunk_id,
                    "text": chunk_text,
                    "token_count": current_tokens
                })
                chunk_id += 1

            # Overlap handling
            overlap = []
            overlap_token_count = 0

            for sent in reversed(current_sentences):
                sent_tokens = count_tokens(sent)
                if overlap_token_count + sent_tokens > overlap_tokens:
                    break
                overlap.insert(0, sent)
                overlap_token_count += sent_tokens

            current_sentences = overlap
            current_tokens = overlap_token_count

        current_sentences.append(sentence)
        current_tokens += sentence_tokens

    # Add final chunk
    if current_tokens >= min_tokens:
        chunks.append({
            "chunk_id": chunk_id,
            "text": " ".join(current_sentences),
            "token_count": current_tokens
        })

    return chunks

def chunk_wikipedia_sentences(sentences: List[str]) -> List[Dict]:
    """
    Public API for Module 2.
    """
    return build_chunks(sentences)


In [ ]:
chunks = chunk_wikipedia_sentences(sentences)

with open("/content/drive/MyDrive/wiki_chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)

print(f"Saved {len(chunks)} chunks")

Saved 6043 chunks


In [ ]:

with open("/content/drive/MyDrive/wiki_chunks.pkl", "rb") as f:
    chunks = pickle.load(f)

print(len(chunks), chunks[0]["token_count"])

6043 589


In [ ]:
print(len(chunks))
print(chunks[0]["token_count"])
print(chunks[0]["text"][:300])

6043
589
April
April (Apr.) is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days. April always begins on the same day of the week as July, and additionally, January in leap years. April always ends on the same day of 


In [ ]:
!pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 76.2 MB/s eta 0:00:00


In [ ]:
from typing import List, Dict, Tuple
import pickle
import numpy as np
import faiss

from sentence_transformers import SentenceTransformer

with open("/content/drive/MyDrive/wiki_chunks.pkl", "rb") as f:
    chunks: List[Dict] = pickle.load(f)

print(f"Loaded {len(chunks)} chunks")


Loaded 6043 chunks


In [ ]:
def load_embedding_model() -> SentenceTransformer:
    """
    Load sentence embedding model.
    """
    model = SentenceTransformer("all-MiniLM-L6-v2")
    return model


In [ ]:
def embed_chunks(
    model: SentenceTransformer,
    chunks: List[Dict],
    batch_size: int = 64
) -> np.ndarray:
    """
    Encode chunk texts into normalized embeddings.
    """
    texts = [c["text"] for c in chunks]
    embeddings = model.encode(
        texts,
        batch_size=batch_size,
        show_progress_bar=True,
        normalize_embeddings=True
    )
    return np.array(embeddings).astype("float32")


In [ ]:
def build_faiss_index(embeddings: np.ndarray) -> faiss.Index:
    """
    Build a cosine-similarity FAISS index.
    """
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    return index


In [ ]:
# Build everything once
embedding_model = load_embedding_model()
chunk_embeddings = embed_chunks(embedding_model, chunks)
index = build_faiss_index(chunk_embeddings)

# Save to Drive
faiss.write_index(index, "/content/drive/MyDrive/wiki_faiss.index")

with open("/content/drive/MyDrive/wiki_chunks_meta.pkl", "wb") as f:
    pickle.dump(chunks, f)

print("FAISS index and metadata saved")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/95 [00:00<?, ?it/s]

FAISS index and metadata saved


In [ ]:
index = faiss.read_index("/content/drive/MyDrive/wiki_faiss.index")

with open("/content/drive/MyDrive/wiki_chunks_meta.pkl", "rb") as f:
    chunks = pickle.load(f)

embedding_model = load_embedding_model()


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
def retrieve_top_k(
    query: str,
    model: SentenceTransformer,
    index: faiss.Index,
    chunks: List[Dict],
    k: int = 3
) -> List[Tuple[Dict, float]]:
    """
    Retrieve top-k most relevant chunks for a query.
    """
    query_embedding = model.encode(
        [query],
        normalize_embeddings=True
    ).astype("float32")

    scores, ids = index.search(query_embedding, k)

    results = []
    for idx, score in zip(ids[0], scores[0]):
        results.append((chunks[idx], float(score)))

    return results


In [ ]:
results = retrieve_top_k(
    query="What is gravity?",
    model=embedding_model,
    index=index,
    chunks=chunks,
    k=3
)

for chunk, score in results:
    print(score)
    print(chunk["text"][:200])
    print("-" * 40)


0.5624241232872009
so it seems natural for astronomers to use the term 'satellite' for these as well. Our Local Group is itself part of an even larger group, the Virgo Supercluster. There are other, even larger, groups 
----------------------------------------
0.5054616332054138
For example, a force causes an affected object to be pushed or pulled in a certain direction. This changes the object's momentum. Forces cause objects to accelerate, add to the object's overall pressu
----------------------------------------
0.4931010901927948
Spacetime is the fact that we have a four-dimensional universe, having three spatial (space) dimensions and one temporal (time) dimension. Any physical event happens at some place inside these three s
----------------------------------------


In [ ]:
!pip install transformers torch

In [ ]:
from typing import List, Dict
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def load_answerer_model(model_name: str):
    """
    Load a small deterministic language model.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    model.eval()
    return tokenizer, model

def build_prompt(question: str, retrieved_chunks: List[Dict]) -> str:
    """
    Build a strict rewriting prompt.
    """
    context = "\n\n".join(chunk["text"] for chunk in retrieved_chunks)

    prompt = (
        "You are an editor.\n"
        "Use ONLY the information in the given text.\n"
        "Do not add facts.\n"
        "If the text does not answer the question, say:\n"
        "\"Not enough information in the Simple Wikipedia dataset.\"\n"
        "Use simple English.\n"
        "Write at most 3 short sentences.\n\n"
        f"Question:\n{question}\n\n"
        f"Text:\n{context}\n\n"
        "Answer:\n"
    )
    return prompt

def build_prompt(question: str, retrieved_chunks: List[Dict]) -> str:
    """
    Build a strict rewriting prompt.
    """
    context = "\n\n".join(chunk["text"] for chunk in retrieved_chunks)

    prompt = (
        "You are an editor.\n"
        "Use ONLY the information in the given text.\n"
        "Do not add facts.\n"
        "If the text does not answer the question, say:\n"
        "\"Not enough information in the Simple Wikipedia dataset.\"\n"
        "Use simple English.\n"
        "Write at most 3 short sentences.\n\n"
        f"Question:\n{question}\n\n"
        f"Text:\n{context}\n\n"
        "Answer:\n"
    )
    return prompt


In [ ]:
def generate_raw_answer(
    question: str,
    retrieved_chunks: List[Dict],
    tokenizer,
    model,
    max_new_tokens: int = 100
) -> str:
    """
    Generate a raw rewritten answer.
    """
    prompt = build_prompt(question, retrieved_chunks)

    inputs = tokenizer(prompt, return_tensors="pt")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the answer part
    answer = decoded.split("Answer:")[-1].strip()
    return answer